In [1]:
import ee,geemap
import numpy as np
Map = geemap.Map()
Map.add_basemap()
sh = ee.Geometry.Rectangle([70, 0, 138, 55], None, False)

In [2]:
#树木覆盖度
vc = ee.Image('projects/ee-lmike18770616551/assets/vc_mean_ee')
# vc.projection().nominalScale()

In [3]:
#干燥指数
pet = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(ee.Date.fromYMD(2001,1,1),ee.Date.fromYMD(2024,1,1)).select('pet').sum().multiply(0.1).multiply(1/23)
pr = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(ee.Date.fromYMD(2001,1,1),ee.Date.fromYMD(2024,1,1)).select('pr').sum().multiply(1/23)
dry = pet.multiply(pr.pow(-1))
# geemap.download_ee_image(dry,r'D:\work\gee_\gee_download\yuan\test\dry.tif',scale= 4638.3,crs= 'EPSG:4326',region=sh)

In [4]:
#多轴图
p_dry = ee.Image(40)
p_dry = p_dry.add(dry.lt(0.5).multiply(-10))
p_dry = p_dry.add(dry.lt(1).multiply(-10))
p_dry = p_dry.add(dry.lt(2).multiply(-10))
p_dry = p_dry.multiply(dry.lt(4))

p_tc = ee.Image(4)
p_tc = p_tc.add(vc.lt(0.25).multiply(-1))
p_tc = p_tc.add(vc.lt(0.5).multiply(-1))
p_tc = p_tc.add(vc.lt(0.75).multiply(-1))
p_tc = p_tc.multiply(vc.lt(1))

p_tc = p_tc.multiply(p_dry.gt(0.5))
p_dry = p_dry.multiply(p_tc.gt(0.5))
p_arr = p_dry.add(p_tc)
# geemap.download_ee_image(p_arr,r'D:\work\gee_\gee_download\yuan\test\p_arr.tif',scale= 4638.3,crs= 'EPSG:4326',region=sh)

In [5]:
import datetime

quans = {}
soil30 = []
star_time = 2001
end_time = 2024

p = 1
for day in range(0, 365, 5):
    swvl_day_list = []
    for year in range(star_time, end_time):
        s = datetime.datetime(year, 1, 1)
        image =  ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate((s+datetime.timedelta(day)).strftime("%Y-%m-%d"), 
                                                                        (s+datetime.timedelta(day+5)).strftime("%Y-%m-%d")).mean()
        swvl = image.expression('swvl1*c1+swvl2*c2+swvl3*c3',
                                 {'swvl1':image.select('soil_temperature_level_1'),
                                  'swvl2':image.select('soil_temperature_level_2'),
                                  'swvl3':image.select('soil_temperature_level_3'),
                                  'c1':0.2333,'c2':0.7,'c3':0.0667})
        swvl_day_list.append(swvl)
    
    swvl_day_images = ee.ImageCollection(swvl_day_list)
    
    for year in range(star_time, end_time):
        ob_im = swvl_day_list[year - star_time]
        quan = swvl_day_images.map(lambda img: ob_im.gt(img)).sum().add(1).divide(end_time-star_time)
        quans[f"{year}-{p}"] = quan
    p+=1

In [11]:
dry_index_year = []

for year in range(2001,2024):
    dry_index = ee.Image(0)
    count = ee.Image(0)
    switch = ee.Image(0)
    st_switch = ee.Image(0)
    st_count = ee.Image(0)
    ed_switch = ee.Image(0)
    ed_count = ee.Image(0)
    for p in range(1,74):
        arr = quans[f'{year}-{p}'].clip(sh)
        st_switch = st_switch.multiply(st_count.lt(5))
        st_count = st_count.multiply(st_switch)
        ed_count = switch.multiply(ed_switch).multiply(-3)
        ed_switch = ed_switch.add(arr.gte(0.2).multiply(switch))
        ed_switch = ed_switch.add(arr.lt(0.2).multiply(switch).multiply(-1))
        ed_count = ed_count.multiply(arr.gte(0.2).multiply(switch.lte(0)))
        st_switch = st_switch.add(arr.gte(0.4))
        st_count = st_count.multiply(arr.gte(0.4))
        st_count = st_count.add(st_switch.gt(0))
        ed_count = ed_count.add(ed_switch.gt(0))
        switch = switch.multiply(ed_count.neq(-2).multiply(ed_switch.lte(0)))
        count = count.add(ed_count.eq(-2).multiply(ed_switch.gt(0)).multiply(-1))
        dry_index = dry_index.add(count.gte(6).multiply(count.lte(18)).multiply(ed_count.eq(-2)).multiply(ed_switch.gt(0)))
        count = count.multiply(ed_count.neq(-2).multiply(ed_switch.lte(0)))
        ed_switch = ed_switch.multiply(ed_count.neq(-2).multiply(ed_switch.lte(0)))
        ed_count = ed_count.multiply(ed_count.neq(-2).multiply(ed_switch.lte(0)))
        switch = switch.add(arr.lt(0.2).multiply(st_switch))
        st_switch = st_switch.multiply(arr.gte(0.2).multiply(st_switch.lte(0)))
        count = count.add(switch)
    dry_index_year.append(dry_index)

dry_index = dry_index_year[0]
for dry_year in dry_index_year[1:]:
    dry_index = dry_index.add(dry_year)

In [15]:
geemap.ee_export_image_to_asset(dry_index, "dry_index", "dry_index", region=sh, scale=10000, crs='epsg:3857')

EEException: Request payload size exceeds the limit: 10485760 bytes.

In [14]:
Map.addLayer(dry_index_year[9], {'max': 10, 'min': 0}, "dry_index")
Map

Map(bottom=7868.0, center=[13.581920900545857, 108.07070931527659], controls=(WidgetControl(options=['position…